## Introduction 

This lesson will cover: 
- Що таке виклик функції та його випадки використання 
- Як створити виклик функції за допомогою OpenAI 
- Як інтегрувати виклик функції в додаток 

## Learning Goals 

After completing this lesson you will know how to and understand: 

-  Мету використання виклику функції 
- Налаштування виклику функції за допомогою сервісу OpenAI 
- Проектування ефективних викликів функцій для випадку використання вашого додатку


## Розуміння викликів функцій

Для цього уроку ми хочемо створити функцію для нашого освітнього стартапу, яка дозволить користувачам використовувати чат-бота для пошуку технічних курсів. Ми рекомендуватимемо курси, які відповідають їхньому рівню навичок, поточній ролі та зацікавленій технології.

Для цього ми використаємо комбінацію:
 - `OpenAI` для створення чат-досвіду для користувача
 - `Microsoft Learn Catalog API` для допомоги користувачам у пошуку курсів на основі запиту користувача
 - `Function Calling` для отримання запиту користувача та передачі його у функцію для виконання API-запиту.

Щоб почати, давайте розглянемо, чому ми взагалі хочемо використовувати виклики функцій:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # отримати нову відповідь від GPT, де він може бачити відповідь функції


print(second_response.choices[0].message)


### Чому Виклик Функцій

Якщо ви пройшли будь-який інший урок цього курсу, ви, ймовірно, розумієте потужність використання Великих Мовних Моделей (LLM). Сподіваюсь, ви також бачите деякі їхні обмеження.

Виклик Функцій — це функція сервісу OpenAI, створена для вирішення наступних проблем:

Непослідовне Форматування Відповідей:
- До впровадження виклику функцій відповіді від великої мовної моделі були неструктурованими та непослідовними. Розробникам доводилося писати складний код валідації для обробки кожного варіанту виводу.

Обмежена Інтеграція з Зовнішніми Даними:
- До цієї функції було складно інтегрувати дані з інших частин застосунку у контекст чату.

Стандартизуючи формати відповідей і забезпечуючи безшовну інтеграцію із зовнішніми даними, виклик функцій спрощує розробку і зменшує потребу в додатковій логіці валідації.

Користувачі не могли отримати відповіді на запитання типу "Яка зараз погода в Стокгольмі?". Це тому, що моделі були обмежені часом, на який були натреновані дані.

Розглянемо приклад нижче, який ілюструє цю проблему:

Припустимо, ми хочемо створити базу даних студентів, щоб пропонувати їм відповідний курс. Нижче наведено два описи студентів, які дуже схожі за даними, що вони містять.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Ми хочемо надіслати це до LLM для парсингу даних. Це пізніше можна буде використати в нашому застосунку для надсилання цього до API або збереження в базі даних.

Давайте створимо два ідентичні запити, в яких ми інструктуємо LLM, яку інформацію нас цікавить:


Ми хочемо надіслати це до LLM, щоб розпарсити частини, які важливі для нашого продукту. Тож ми можемо створити два ідентичні підказки для інструктажу LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Після створення цих двох підказок ми надішлемо їх до LLM за допомогою `openai.ChatCompletion`. Ми зберігаємо підказку у змінній `messages` і призначаємо роль `user`. Це імітує повідомлення від користувача, яке надсилається чат-боту.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Тепер ми можемо надіслати обидва запити до LLM і проаналізувати отриману відповідь.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Навіть якщо підказки однакові, а описи схожі, ми можемо отримати різні формати властивості `Grades`.

Якщо запустити наведений вище блок кілька разів, формат може бути `3.7` або `3.7 GPA`.

Це тому, що LLM приймає неструктуровані дані у вигляді написаної підказки і також повертає неструктуровані дані. Нам потрібен структурований формат, щоб знати, чого очікувати при збереженні або використанні цих даних.

Використовуючи функціональний виклик, ми можемо переконатися, що отримуємо структуровані дані у відповідь. При використанні функціонального виклику LLM фактично не виконує жодних функцій. Натомість ми створюємо структуру, якій LLM має слідувати у своїх відповідях. Потім ми використовуємо ці структуровані відповіді, щоб знати, яку функцію запускати у наших додатках.


![Діаграма потоку виклику функції](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.uk.png)


Ми можемо взяти те, що повертається з функції, і надіслати це назад до LLM. LLM тоді відповість природною мовою, щоб відповісти на запит користувача.


### Випадки використання викликів функцій

**Виклик зовнішніх інструментів**  
Чатботи чудово підходять для надання відповідей на запитання користувачів. Використовуючи виклики функцій, чатботи можуть використовувати повідомлення від користувачів для виконання певних завдань. Наприклад, студент може попросити чатбота "Надіслати електронного листа моєму викладачу з проханням про додаткову допомогу з цього предмета". Це може викликати функцію `send_email(to: string, body: string)`.

**Створення запитів до API або бази даних**  
Користувачі можуть знаходити інформацію, використовуючи природну мову, яка перетворюється у форматований запит або API-запит. Прикладом може бути вчитель, який запитує "Хто з учнів виконав останнє завдання", що може викликати функцію з назвою `get_completed(student_name: string, assignment: int, current_status: string)`.

**Створення структурованих даних**  
Користувачі можуть взяти блок тексту або CSV і використати LLM для вилучення важливої інформації з нього. Наприклад, студент може конвертувати статтю з Вікіпедії про мирні угоди для створення AI-флешкарток. Це можна зробити за допомогою функції `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. Створення вашого першого виклику функції

Процес створення виклику функції включає 3 основні кроки:  
1. Виклик API Chat Completions зі списком ваших функцій та повідомленням користувача  
2. Читання відповіді моделі для виконання дії, тобто виклику функції або API  
3. Здійснення ще одного виклику до API Chat Completions з відповіддю вашої функції, щоб використати цю інформацію для створення відповіді користувачу.


![Потік виклику функції](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.uk.png)


### Елементи виклику функції 

#### Вхідні дані користувача 

Першим кроком є створення повідомлення користувача. Його можна динамічно призначити, взявши значення з текстового поля, або ви можете призначити значення тут. Якщо ви вперше працюєте з API Chat Completions, нам потрібно визначити `role` та `content` повідомлення. 

`role` може бути `system` (створення правил), `assistant` (модель) або `user` (кінцевий користувач). Для виклику функції ми призначимо це як `user` і приклад запитання. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Створення функцій.

Далі ми визначимо функцію та параметри цієї функції. Тут ми використаємо лише одну функцію під назвою `search_courses`, але ви можете створювати кілька функцій.

**Важливо** : Функції включені в системне повідомлення для LLM і будуть враховані у кількості доступних токенів, які у вас є.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Визначення** 

Структура визначення функції має кілька рівнів, кожен із яких має власні властивості. Ось розбір вкладеної структури:

**Властивості функції верхнього рівня:**

`name` - Назва функції, яку ми хочемо викликати. 

`description` - Опис того, як працює функція. Тут важливо бути конкретним і чітким. 

`parameters` - Список значень і формату, які ви хочете, щоб модель генерувала у своїй відповіді. 

**Властивості об’єкта параметрів:**

`type` - Тип даних об’єкта параметрів (зазвичай "object")

`properties` - Список конкретних значень, які модель використовуватиме у своїй відповіді. 

**Властивості окремих параметрів:**

`name` - Неявно визначається ключем властивості (наприклад, "role", "product", "level")

`type` - Тип даних цього конкретного параметра (наприклад, "string", "number", "boolean") 

`description` - Опис конкретного параметра. 

**Необов’язкові властивості:**

`required` - Масив, у якому перелічено, які параметри є обов’язковими для виконання виклику функції. 


### Виклик функції  
Після визначення функції, тепер нам потрібно включити її у виклик API Chat Completion. Ми робимо це, додаючи `functions` до запиту. У цьому випадку `functions=functions`.  

Також є опція встановити `function_call` у `auto`. Це означає, що ми дозволимо LLM самостійно вирішити, яку функцію слід викликати на основі повідомлення користувача, замість того, щоб призначати це самостійно.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Тепер давайте подивимося на відповідь і побачимо, як вона відформатована:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Ви можете побачити, що викликається ім'я функції, і з повідомлення користувача LLM змогла знайти дані, які відповідають аргументам функції.


## 3.Інтеграція викликів функцій у додаток. 


Після того, як ми протестували відформатовану відповідь від LLM, тепер ми можемо інтегрувати це у додаток. 

### Керування потоком 

Щоб інтегрувати це у наш додаток, давайте виконаємо наступні кроки: 

Спочатку зробимо виклик до сервісів OpenAI і збережемо повідомлення у змінній під назвою `response_message`. 


In [ ]:
response_message = response.choices[0].message

Тепер ми визначимо функцію, яка викликатиме API Microsoft Learn для отримання списку курсів:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Як найкраща практика, ми потім перевіримо, чи модель хоче викликати функцію. Після цього ми створимо одну з доступних функцій і зіставимо її з функцією, яку викликають.  
Потім ми візьмемо аргументи функції і зіставимо їх з аргументами з LLM.

Нарешті, ми додамо повідомлення про виклик функції та значення, які були повернені повідомленням `search_courses`. Це дає LLM всю необхідну інформацію для  
відповіді користувачу природною мовою.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Тепер ми надішлемо оновлене повідомлення до LLM, щоб отримати відповідь природною мовою замість відповіді у форматі JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Виклик коду

Чудова робота! Щоб продовжити навчання з OpenAI Function Calling, ви можете створити: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Більше параметрів функції, які можуть допомогти учням знайти більше курсів. Ви можете знайти доступні параметри API тут:  
 - Створіть ще один виклик функції, який приймає більше інформації від учня, наприклад їх рідну мову  
 - Створіть обробку помилок, коли виклик функції та/або виклик API не повертає жодних відповідних курсів  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується звертатися до професійного людського перекладу. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
